# Attacks on Neural Networks in a Lightweight Speech Pseudonymization Pipeline

## Imports

In [1]:
import ASR_2024_anonymization_module_learning.speaker_anonymization as pipeline
import ASR_2024_anonymization_module_learning.main as pipeline_optimization

import util
from backdoored_dataset import BackdooredVCTK
from neural_model import NeuralModel
from attacks.jingleback import JingleBack
from metrics import attack_success_rate, clean_accuracy_drop

import torch

from torchattacks.attacks.fgsm import FGSM
from torchattacks.attacks.pgd import PGD

c:\Users\Daan\Documents\Projecten\ru-automatic-speech-recognition-23-24\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda:0


## Preparation

In [2]:
TARGET_LABEL = 0

pipeline_config = pipeline.config.Config(
    num_trials=1,
    n_speakers=10,
    n_samples_per_speaker=10,
    gender=None,
    min_age=None,
    max_age=None,
    accent=None,
    region=None
)

util.set_global_seed(3131)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

if str(device) == "cuda":
    print(torch.cuda.get_device_name())

cuda
NVIDIA GeForce GTX 1050


## Attacks
### Backdoor Attacks: JingleBack

In [3]:
jingleback_attack = JingleBack(source_label=None, target_label=TARGET_LABEL)
# model = NeuralModel(attack=jingleback_attack, device=device)

backdoored_dataset = BackdooredVCTK(jingleback_attack, epsilon=0.05, train=True, pipeline_config=pipeline_config)

2024-06-03 16:07:32,226 - INFO - Loading data from cache: d:/Datasets/vctk/cache\cache_10_10_None_None_None_None_None.pkl


{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
{'This is a very common type of bow, one showing mainly red and yellow, with little or no green or blue.', 'Two other men, including the taxi driver, were wounded in the attack.', 'It is the whole package.', 'Mark Fisher was a guest of the Northern Ireland Tourist Board.', 'The Norsemen considered the rainbow as a bridge over which the gods passed from earth to their home in the sky.', 'We do not expect any surplus.', 'They are now divorced.', 'He was popular.', 'We will be homeless.', 'The Tory party will last.', 'The group is separate from the Real IRA.', 'Indeed, it is only the beginning.', 'I saw it at a ruck.', 'A spokeswoman for Edinburgh City Council confirmed its support for the company.', 'The procedure is the same in all cases.', 'However, it has to be seen as part of a package.', 'But it might have done.', 'As it was a double, it was pretty expensive.', 'It is not for any other purpose.', 'Here, again, they were denied a voice.', 'A web site is

KeyboardInterrupt: 

### Evasion Attacks: FGSM & PGD

In [ ]:
fgsm = FGSM(model.neural_network, eps=8/255)
fgsm.set_mode_targeted_by_label() #NOTE: This means that, when attacking the model, you should pass the target label manually/yourself. So fgsm(audio, target_label).

In [ ]:
fgsm = PGD(model.neural_network, eps=8/255, alpha=2/255, steps=10, random_start=True)
fgsm.set_mode_targeted_by_label() #NOTE: This means that, when attacking the model, you should pass the target label manually/yourself. So fgsm(audio, target_label).

## Pipeline

In [ ]:
configs = [pipeline_config]
pipeline_optimization.run_optimization(pipeline_config)

asv_model = pipeline.spi.SpeakerIdentificationModel() # ASV model
asv_model.finetune_model() # Method for finetuning ASV model during initialisation